In [1]:
import numpy as np
import matplotlib.pyplot as plt

from popai import parse_input
from popai import process_empirical
from popai import generate_models
from popai import simulate_data
from popai import build_predictors

2024-05-17 09:25:44.321303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# popai walk-through

This jupyter notebook walks through each step in running popai, including parsing and reading input data, building SFS, building models, simulating data, training machine learning models, and applying models to empirical data.

## Step 1: Parse the configuration file

We will use the ModelConfigParser class to parse the information from the configuration. A full description of the configuration file, along with an example, is available [here](https://popai.readthedocs.io/en/latest/usage/parsinginput.html). This file provides popai with the information it needs to conduct a species delimitation analysis. 

In [ ]:
config_parser = parse_input.ModelConfigParser("../input_files/config.txt")
config_values = config_parser.parse_config()

## Step 2: Process empirical data

Next, we will process our empirical data. The format for the empirical data is described in detail [here](https://delimitpy.readthedocs.io/en/latest/usage/parsinginput.html). 

There are three major steps:
1) Read data from fasta files and convert to a numpy array.
2) Choose values for down-projecting the Site Frequency Spectrum.
3) Build SFS with down-projection.
4) Determine the average number of SNPs used to construct the SFS.


### Step 2a: Read data into numpy array

In [ ]:
data_processor = process_empirical.DataProcessor(config=config_values)
empirical_array = data_processor.fasta_to_numpy()
print(f"Our input data has {empirical_array.shape[0]} individuals and {empirical_array.shape[1]} SNPs.")

### Step 2b: Choose values for down-projection

SFS cannot be generated from datasets that include missing data. To circumvent this, we use a downsampling approach such as that described in Satler and Carstens [(2017)](https://doi.org/10.1111/mec.14137). We must choose thresholds for each populations (i.e., the minumum number of individuals that must be sampled for a SNP to be used.) To help with this, we use the function find_downsampling from the class DataProcessor. This function generates a dictionary that holds the number of SNPs that meet each threshold.

We will use a folded SFS, meaning that we will build the SFS based on minor allele frequencies.

Since our data should be phased, and we will simulate diploid individuals, we will only consider multiples of 2.

In [ ]:
# generate dictionary with the number of SNPs at different sampling thresholds
empirical_2d_sfs_sampling = data_processor.find_downsampling(empirical_array)

# print threshold, SNP combos for thresholds with at least 1030 snps
minspns = 1030
min_filtered = {key: value for key, value in empirical_2d_sfs_sampling.items() if value >= minspns}
print(min_filtered)


### Step 2c: Build SFS with down-projection

We want to maximize the number of individuals and SNPs we can use. In this tutorial, since the input data were simulated without missing data, we can use all individuals and still retain all the SNPs.

Now, we are ready to build our SFS. We are only using a single replicate for this test. This makes sense because our ‘empirical’ data are actually simulated data, and we are not downsampling. Because of this, we do not expect much noise. For messier empirical data, use ~10 reps and ensure that results do not differ across replicates.

We will build a set of joint SFS (jSFS) between each pairwise combination of populations. Additionally, we will build a multidimensional SFS (mSFS). When building the mSFS, we can optionally choose to use a binning approach [(Smith et al., 2018)](https://doi.org/10.1111/mec.14223). This creates a coarser SFS, which can be helpful if we have a lot of individuals and not that many SNPs.

The code below will also print the number of SNPs used on average to construct the mSFS, which will be helpful information when deciding how many SNPs to use to build the SFS for the simulated data.

In [ ]:
downsampling_dictionary = {"A":20, "B":20, "C":20}

# build 10 replicates of the 2d SFS
empirical_2d_sfs = data_processor.numpy_to_2d_sfs(empirical_array, downsampling=downsampling_dictionary, replicates = 1)

# build 10 replicates of the mSFS
empirical_msfs, average_snps = data_processor.numpy_to_msfs(empirical_array, downsampling=downsampling_dictionary, replicates = 1)

## Step 3: Build the models, and draw parameters from priors

Next, we will use functions from the ModelBuilder class to generate a baseline set of models (without parameters). Then, we use draw_parameters to draw parameterized models from the parameter space. This will return a lists of lists. Each list will correspond to a model. For each model, we have a list of parameterized versions of that model, with parameters drawn from the priors defined by the user in the configuration and species tree files.

A full description of the models generated by popai is available [here](https://popai.readthedocs.io/en/latest/usage/buildingmodels.html).

In [ ]:
# build the models
model_builder = generate_models.ModelBuilder(config_values)
divergence_demographies, sc_demographies, dwg_demographies = model_builder.build_models()

# parameterize the models
parameterized_models, labels = model_builder.draw_parameters(divergence_demographies, sc_demographies, dwg_demographies)

print((f"Number of models: {len(parameterized_models)}"))
print((f"Number of replicates: {len(parameterized_models[0])}"))

## Step 4: Validate the models by  plotting

Now, we must ask "Are these the models I'm looking for?"

One way to easily check this is by plotting the models. The validate_models function takes as input your lists returned from the parameterization command above (draw_parameters). For each model, it will draw one parameterization at random, and plot it using functionality from the drawdemes package.

Make sure the models look like you hoped they would! Remember, you are only seeing a single parameterization for each model.

In [ ]:
model_builder.validate_models(parameterized_models, labels)

## Step 5: Simulate Data

Now that we have processed our empirical data and build our models, we are ready to simulate the data that we will use to train our machine learning algorithms. We will use information from processing our empirical data (the number of individuals and SNPs to use when bulding the SFS).

In [ ]:
data_simulator = simulate_data.DataSimulator(parameterized_models, labels, config=config_values, cores=1, downsampling=downsampling_dictionary, max_sites = average_snps)

arrays, labels = data_simulator.simulate_ancestry() # simulate ancestry in msprime

## Step 6: Build SFS from simulated data

Now, we will create a numpy array and build jSFS and mSFS from our simulated data.

In [ ]:
mSFS = data_simulator.mutations_to_sfs(arrays) # generate mSFS
jSFS = data_simulator.mutations_to_2d_sfs(arrays) # generate 2D SFS

## Step 7: Train Classifiers

We now have our simulated training data and are ready to train our classifiers. We will train three classifiers.

1) A Random Forest classifier that takes as input the bins of the multidimensional SFS (mSFS).

2) A Fully Connected Neural Network that takes as input the bins of the multidimensional SFS (mSFS).

3) A Convolutional Neural Network that takes as input the jSFS between all pairs of populations.

### Step 7a: Train the RF classifier

The code will return the model and the confusion matrix and will print the out-of-bag error rates and the confusion matrix on the validation data.

In [ ]:
random_forest_sfs_predictor = build_predictors.RandomForestsSFS(config_values, mSFS, labels)
random_forest_sfs_model, random_forest_sfs_cm, random_forest_sfs_cm_plot = random_forest_sfs_predictor.build_rf_sfs()
random_forest_sfs_cm_plot.show()

### Step 7b: Train the FCNN

In [ ]:
neural_network_sfs_predictor = build_predictors.NeuralNetSFS(config_values, mSFS, labels)
neural_network_sfs_model, neural_network_sfs_cm, neural_network_sfs_cm_plot = neural_network_sfs_predictor.build_neuralnet_sfs()
neural_network_sfs_cm_plot.show()

### Step 7c: Train the CNN

In [ ]:
cnn_2d_sfs_predictor = build_predictors.CnnSFS(config_values, jSFS, labels)
cnn_2d_sfs_model, cnn_2d_sfs_cm, cnn_2d_sfs_cm_plot = cnn_2d_sfs_predictor.build_cnn_sfs()
cnn_2d_sfs_cm_plot.show()

## Step 8: Now, we are ready to apply our classifiers to our empirical data.

We will use the three classifiers trained above to make predictions on our empirical data.

### Step 8a: Random Forest predictions

In [ ]:
results_rf = random_forest_sfs_predictor.predict(random_forest_sfs_model, empirical_msfs)
print(results_rf)

### Step 8b: FCNN predictions

In [ ]:
results_fcnn = neural_network_sfs_predictor.predict(neural_network_sfs_model, empirical_msfs)
print(results_fcnn)

### Step 8c: CNN predictions

In [ ]:
results_cnn = cnn_2d_sfs_predictor.predict(cnn_2d_sfs_model, empirical_2d_sfs)
print(results_cnn)